In [1]:
import numpy as np
import matplotlib as plt

class coin_model:
    def __init__(self, p, q):
        self.coins_H_p = [p, q]
    def throw_coin(self, coin_num):
        p = self.coins_H_p[coin_num]
        val = np.random.random()
        if val <= p:
            return 1
        return 0
    """ Generate the sample data for the current distribution.
    """
    def generate_sample(self, times, default_per_times=10):
        sample = np.zeros((times, default_per_times+1), dtype=int)
        for i in range(times):
            a = 0 if np.random.random() <= 0.5 else 1
            for j in range(default_per_times+1):
                sample[i][j] = self.throw_coin(a)
            sample[i][default_per_times] = a
        return sample

The following is just a test for the result from above.  
The last column indicate which coin we are throwing.

In [2]:
c = coin_model(0.3, 0.6)
sample = c.generate_sample(1000)
# print(sample)
em_sample = sample[:, :-1]
print(em_sample)

# em_sample.shape

[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 1 1]
 [0 0 0 ... 1 0 1]
 ...
 [1 0 1 ... 0 1 1]
 [0 0 0 ... 1 1 1]
 [1 0 1 ... 1 0 0]]


Now, we want to apply the EM Algorithm to this model. We want to guess the p(H|coin=0) and p(H|coin=1). But imagine if we don't konw which coin we are throwing each row, then we have to guess.

According to the EM Algorithm, we want to maximize $$\mathcal{l}(x|\theta) = \log P(x|\theta)$$  
By definition, we can see that
\begin{align*}
\log P(x|\theta)
&= \log \prod_{i=1}^m P(x_i|\theta)\\
&= \sum_{i=1}^m \log P(x_i|\theta)\\
&= \sum_{i=1}^m \log \sum_c P(x_i, c|\theta)\\
&= \sum_{i=1}^m \log \sum_c Q_i(c)\cdot \frac{P(x_i, c|\theta)}{Q_i(c)}\\
&\geq \sum_{i=1}^m \sum_c Q_i(c) \log \frac{P(x_i, c|\theta)}{Q_i(c)} \quad\text{By $\log(E(x))\geq E(\log(x))$}
\end{align*}
Now, consider $\frac{P(x_i, c|\theta)}{Q_i(c)}$, if we can make it be a function that isn't influenced by c, we would have the equality. So let $Q_i(c)=P(c|x_i,\theta)$, we can see that $$\frac{P(x_i, c|\theta)}{Q_i(c)} = P(x_i|\theta)$$
We can use the Bayes' Rule to calculate $Q_i(c)$
\begin{align*}
Q_i(c)
&= P(c|x_i,\theta)\\
&= \frac{P(c, x_i|\theta)}{P(x_i)}\\
&= \frac{P(c, x_i|\theta)}{\sum_c P(x_i, c|\theta)}\\
&= \frac{P(x_i|c, \theta)P(c|\theta)}{\sum_c P(x_i|c, \theta)P(c|\theta)}
\end{align*}
The above is called E step

Now, for the M step, we just want to maximize the function $F(\theta)$: 
\begin{align*}
\sum_{i=1}^m \sum_c Q_i(c) \log \frac{P(x_i, c|\theta)}{Q_i(c)}
&= \sum_{i=1}^m \sum_c Q_i(c) \log \frac{P(x_i|c,\theta)P(c|\theta)}{Q_i(c)}\\
&= \sum_{i=1}^m \sum_c Q_i(c) \log \frac{P(H|c;\theta)^{n_i(H)}P(T|c;\theta)^{n_i(T)}P(c|\theta)}{Q_i(c)}
\end{align*}
We also have the following restriction:
\begin{align*}
P(H|c,\theta) + P(T|c,\theta) = 1
\end{align*}

Then By Lagrange Theorem, we would have
\begin{align*}
\nabla F_{P(H|c;\theta)} &= \sum_{i=1}^m \frac{Q_i(c)n_i(H)}{P(H|c;\theta)}\\
\nabla F_{P(T|c;\theta)} &= \sum_{i=1}^m \frac{Q_i(c)n_i(T)}{P(T|c;\theta)}\\
\begin{bmatrix}
\nabla F_{P(H|c;\theta)}\\
\nabla F_{P(T|c;\theta)}
\end{bmatrix}
&=
\begin{bmatrix}
\lambda\\
\lambda
\end{bmatrix}
\end{align*}
Then we will have
\begin{align*}
P(H|c;\theta), P(T|c;\theta) &= \sum_{i=1}^m\frac{Q_i(c)n_i(H)}{\lambda}, \sum_{i=1}^m\frac{Q_i(c)n_i(T)}{\lambda}\\
\sum_{i=1}^m\frac{Q_i(c)n_i(H)}{\lambda} + \sum_{i=1}^m\frac{Q_i(c)n_i(T)}{\lambda} &= 1\\
\lambda &= \sum_{i=1}^m Q_i(c)n_i(H) + \sum_{i=1}^m Q_i(c)n_i(T)
\end{align*}

In [3]:
a = np.array([[1, 2, 3]])
b = np.array([[4, 5, 6]])
np.hstack((a, b))

array([[1, 2, 3, 4, 5, 6]])

In [4]:
"""
Return a nxm array, n represents the times of experiments, m is the number of coins   
"""
def E_Step(theta, sample):
    exp_times, per_exp_times = sample.shape
    per_exp_times = np.array([per_exp_times for i in range(exp_times)])
    h_times = np.sum(sample, axis=1)
#     print("per_exp_times is: {}\nh_times is: {}".format(per_exp_times, h_times))
    p_0_H, p_1_H = theta
    p_x_0 = (p_0_H**h_times) * ((1-p_0_H)**(per_exp_times-h_times))
    p_x_1 = (p_1_H**h_times) * ((1-p_1_H)**(per_exp_times-h_times))
    # Notice here we assume P(c|theta) be 0.5, then we can just ignore this in Q_i(c)
    Q_0 = p_x_0 / (p_x_0 + p_x_1)
    Q_1 = p_x_1 / (p_x_0 + p_x_1)
    Q = np.vstack((Q_0, Q_1)).T
    return Q

"""
Update theta
"""
def M_Step(Q, sample):
    exp_times, per_exp_times = sample.shape
    per_exp_times = np.array([per_exp_times for i in range(exp_times)])
    h_times = np.sum(sample, axis=1)
    t_times = per_exp_times - h_times
    h_times = np.vstack((h_times, h_times)).T
    t_times = np.vstack((t_times, t_times)).T
    Q_c_n_H = np.multiply(Q, h_times)
    Q_c_n_T = np.multiply(Q, t_times)
    sum_Q_c_n_H = np.sum(Q_c_n_H, axis=0)
    sum_Q_c_n_T = np.sum(Q_c_n_T, axis=0)
    P_H_c = sum_Q_c_n_H / (sum_Q_c_n_H + sum_Q_c_n_T)
    return P_H_c

In [5]:
theta = [0.2, 0.3]
d = 1
while d > 0.000000000000000000001:
    Q = E_Step(theta, em_sample)
    theta_prev = theta
    theta = M_Step(Q, em_sample)
    d = np.linalg.norm(theta - theta_prev)
print(theta)

[0.30795467 0.60455125]
